In [1]:
import pandas as pd
import numpy as np
from googletrans import Translator  

In [2]:
df = pd.read_csv("competition_dataset/dev.tsv", sep="\t")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120744 entries, 0 to 120743
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   country      120739 non-null  object 
 1   description  120744 non-null  object 
 2   designation  84226 non-null   object 
 3   province     120739 non-null  object 
 4   region_1     100736 non-null  object 
 5   region_2     48736 non-null   object 
 6   variety      120744 non-null  object 
 7   winery       120744 non-null  object 
 8   quality      120744 non-null  float64
dtypes: float64(1), object(8)
memory usage: 8.3+ MB


In [4]:
df

,country,description,designation,province,region_1,region_2,variety,winery,quality
0,France,"A creamed pear wine, with an attractive tang o...",Brut Blanc de Blancs,Alsace,Crémant d'Alsace,NaN,Pinot Blanc,Lucien Albrecht,45.0
1,US,"Simple and dry, this Cabernet has modest black...",NaN,California,Paso Robles,Central Coast,Cabernet Sauvignon,Castle Rock,31.0
2,US,"This lovely wine captures the floral, perfumed...",NaN,Oregon,Willamette Valley,Willamette Valley,Gewürztraminer,Château Bianca,35.0
3,Portugal,"The aromas are the thing here, as so often wit...",NaN,Alentejano,NaN,NaN,Touriga Nacional,Herdade do Esporão,41.0
4,Italy,"This is an interesting, outright strange wine ...",Natì,Southern Italy,Pompeiano,NaN,Coda di Volpe,Sorrentino,37.0
...,...,...,...,...,...,...,...,...,...
120739,Italy,This is a lovely Sauvignon with very character...,NaN,Northeastern Italy,Friuli,NaN,Sauvignon,Plozner,40.0
120740,Spain,"Opens somewhat clipped and choppy, with citrus...",Reserva,Northern Spain,Rioja,NaN,Tempranillo Blend,Ramón Bilbao,41.0
120741,Australia,Surprisingly complete and harmonious for a Rie...,NaN,South Australia,Adelaide Hills,NaN,Riesling,Wagtail,26.0
120742,Germany,Broad aromas of ripe yellow fruit and a hint o...,Spätlese,Mosel,NaN,NaN,Riesling,Pamona Cellars,26.0


In [5]:
#with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
print(df["designation"].value_counts())

Reserve                         2189
Reserva                         1436
Estate                          1245
Barrel sample                   1069
Riserva                          609
                                ... 
Pur Chardonnay                     1
Sonoma County                      1
Loner W11-B                        1
Cemetery Vineyard Indigenous       1
Mezzan                             1
Name: designation, Length: 27800, dtype: int64


In [6]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(df["designation"].value_counts()[:100])

Reserve                      2189
Reserva                      1436
Estate                       1245
Barrel sample                1069
Riserva                       609
Barrel Sample                 505
Brut                          483
Crianza                       398
Estate Grown                  362
Estate Bottled                318
Dry                           303
Old Vine                      277
Gran Reserva                  263
Brut Rosé                     208
Extra Dry                     194
Vieilles Vignes               179
Bien Nacido Vineyard          152
Rosé                          141
Late Bottled Vintage          136
Réserve                       136
Unoaked                       134
Single Vineyard               126
Late Harvest                  125
Tradition                     117
Vintage                       116
Barrel Select                 113
Grand Reserve                 107
Classic                       101
Estate Reserve                 98
Tinto         

In [5]:
#df[df["designation_tr"] == "reserva" | df["designation_tr"] == "riserva" | df["designation_tr"] == "rèserve"]["designation_tr"] = "reserve"
df["designation_tr"] = df["designation"].apply(lambda d: d.lower() if pd.isna(d) == False else d)
df["designation_tr"] = df["designation_tr"].replace(["reserva","riserva","rèserve"],["reserve","reserve","reserve"])


In [8]:
df

,country,description,designation,province,region_1,region_2,variety,winery,quality,designation_tr
0,France,"A creamed pear wine, with an attractive tang o...",Brut Blanc de Blancs,Alsace,Crémant d'Alsace,NaN,Pinot Blanc,Lucien Albrecht,45.0,brut blanc de blancs
1,US,"Simple and dry, this Cabernet has modest black...",NaN,California,Paso Robles,Central Coast,Cabernet Sauvignon,Castle Rock,31.0,NaN
2,US,"This lovely wine captures the floral, perfumed...",NaN,Oregon,Willamette Valley,Willamette Valley,Gewürztraminer,Château Bianca,35.0,NaN
3,Portugal,"The aromas are the thing here, as so often wit...",NaN,Alentejano,NaN,NaN,Touriga Nacional,Herdade do Esporão,41.0,NaN
4,Italy,"This is an interesting, outright strange wine ...",Natì,Southern Italy,Pompeiano,NaN,Coda di Volpe,Sorrentino,37.0,natì
...,...,...,...,...,...,...,...,...,...,...
120739,Italy,This is a lovely Sauvignon with very character...,NaN,Northeastern Italy,Friuli,NaN,Sauvignon,Plozner,40.0,NaN
120740,Spain,"Opens somewhat clipped and choppy, with citrus...",Reserva,Northern Spain,Rioja,NaN,Tempranillo Blend,Ramón Bilbao,41.0,reserve
120741,Australia,Surprisingly complete and harmonious for a Rie...,NaN,South Australia,Adelaide Hills,NaN,Riesling,Wagtail,26.0,NaN
120742,Germany,Broad aromas of ripe yellow fruit and a hint o...,Spätlese,Mosel,NaN,NaN,Riesling,Pamona Cellars,26.0,spätlese


In [6]:
# rimuovo regioni doppie e creo regione unica
df.loc[df["region_1"] == df["region_2"], "region_2"] = ""
df["region_complete"] = df[["region_1","region_2"]].fillna(" ").agg(' '.join, axis=1)

,country,description,designation,province,region_1,region_2,variety,winery,quality,designation_tr,region_complete
0,France,"A creamed pear wine, with an attractive tang o...",Brut Blanc de Blancs,Alsace,Crémant d'Alsace,NaN,Pinot Blanc,Lucien Albrecht,45.0,brut blanc de blancs,Crémant d'Alsace
1,US,"Simple and dry, this Cabernet has modest black...",NaN,California,Paso Robles,Central Coast,Cabernet Sauvignon,Castle Rock,31.0,NaN,Paso Robles Central Coast
2,US,"This lovely wine captures the floral, perfumed...",NaN,Oregon,Willamette Valley,,Gewürztraminer,Château Bianca,35.0,NaN,Willamette Valley
3,Portugal,"The aromas are the thing here, as so often wit...",NaN,Alentejano,NaN,NaN,Touriga Nacional,Herdade do Esporão,41.0,NaN,
4,Italy,"This is an interesting, outright strange wine ...",Natì,Southern Italy,Pompeiano,NaN,Coda di Volpe,Sorrentino,37.0,natì,Pompeiano
...,...,...,...,...,...,...,...,...,...,...,...
120739,Italy,This is a lovely Sauvignon with very character...,NaN,Northeastern Italy,Friuli,NaN,Sauvignon,Plozner,40.0,NaN,Friuli
120740,Spain,"Opens somewhat clipped and choppy, with citrus...",Reserva,Northern Spain,Rioja,NaN,Tempranillo Blend,Ramón Bilbao,41.0,reserve,Rioja
120741,Australia,Surprisingly complete and harmonious for a Rie...,NaN,South Australia,Adelaide Hills,NaN,Riesling,Wagtail,26.0,NaN,Adelaide Hills
120742,Germany,Broad aromas of ripe yellow fruit and a hint o...,Spätlese,Mosel,NaN,NaN,Riesling,Pamona Cellars,26.0,spätlese,


In [7]:
singular_region_complete = set(df["region_complete"])
len(singular_region_complete)

1211

In [12]:
import geocoder

g = geocoder.arcgis('Redlands, CA')
print(g.latlng)

[34.05538000000007, -117.18258999999995]


In [18]:
import geocoder

singular_region_complete_with_latlong = []
for region in singular_region_complete:
    print(region)
    loc =  geocoder.arcgis(region,maxRows=1)
    print(loc)
    try:
        singular_region_complete_with_latlong.append((region, loc.latlng[0],loc.latlng[1]))   
    except:
        pass

Bolgheri  
<[OK] Arcgis - Geocode [Bolgheri, Castagneto Carducci, Livorno]>
Alicante  
<[OK] Arcgis - Geocode [Alicante, Comunidad Valenciana]>
Palette  
<[OK] Arcgis - Geocode [Palette, Le Tholonet, Bouches-du-Rhône, Provence-Alpes-Côte d'Azur]>
McLaren Vale-Padthaway  
<[OK] Arcgis - Geocode [Padthaway, South Australia]>
Chianti Colli Pisani  
<[OK] Arcgis - Geocode [Pisani, Lages, Santa Catarina]>
Strathbogie Ranges  
<[OK] Arcgis - Geocode [Strathbogie Ranges]>
Vin de Pays de France  
<[OK] Arcgis - Geocode [Pays, Monein, Pyrénées-Atlantiques, Nouvelle-Aquitaine]>
Wrattonbully  
<[OK] Arcgis - Geocode [Wrattonbully, South Australia]>
Bourgogne Vézelay  
<[OK] Arcgis - Geocode [Vézelay, Yonne, Bourgogne-Franche-Comté]>
Napa County-Sonoma County Napa-Sonoma
<[OK] Arcgis - Geocode [Sonoma County, California]>
Galluccio  
<[OK] Arcgis - Geocode [Galluccio, Caserta]>
Pouilly-Vinzelles  
<[OK] Arcgis - Geocode [Rue Pouilly-Vinzelles, 71000, Mâcon, Saône-et-Loire, Bourgogne-Franche-Comté]

<[OK] Arcgis - Geocode [Southeastern New England Dental Group]>
Latricières-Chambertin  
<[OK] Arcgis - Geocode [Chambertin, Boucé, Allier, Auvergne-Rhône-Alpes]>
Mendocino-Sonoma-Amador California Other
<[OK] Arcgis - Geocode [Mendocino, California]>
Beaujolais Rosé  
<[OK] Arcgis - Geocode [Beaujolais]>
Côtes Catalanes  
<[OK] Arcgis - Geocode [Côte Catalane]>
Vin de Pays des Coteaux de L'Ardeche  
<[OK] Arcgis - Geocode [Impasse des Coteaux, 07110, Largentière, Ardèche, Auvergne-Rhône-Alpes]>
Adelaide Hills  
<[OK] Arcgis - Geocode [Adelaide Hills, South Australia]>
Bâtard-Montrachet  
<[OK] Arcgis - Geocode [Montrachet, Ridgeland, Mississippi]>
Saint-Amour  
<[OK] Arcgis - Geocode [Saint-Amour, Jura, Bourgogne-Franche-Comté]>
Cadillac Côtes de Bordeaux  
<[OK] Arcgis - Geocode [Cadillac]>
Verdicchio dei Castelli di Jesi Classico Superiore  
<[OK] Arcgis - Geocode [Castelli, Chaco]>
Mediterranée  
<[OK] Arcgis - Geocode [Mediterranee]>
Friuli  
<[OK] Arcgis - Geocode [Friuli]>
Périg

<[OK] Arcgis - Geocode [Valdeorras]>
San Rafael  
<[OK] Arcgis - Geocode [San Rafael, Mendoza]>
Central Victoria  
<[OK] Arcgis - Geocode [Central Victoria, Quintana Roo]>
San Francisco Bay-Livermore Valley Central Coast
<[OK] Arcgis - Geocode [Livermore Cmn, Fremont, California, 94539]>
Biferno Rosso  
<[OK] Arcgis - Geocode [Rosso, Saint-Louis]>
Saint-Véran  
<[OK] Arcgis - Geocode [Saint-Véran, Beaumes-de-Venise, Vaucluse, Provence-Alpes-Côte d'Azur]>
Alto Adige Valle Isarco  
<[OK] Arcgis - Geocode [La Valle, Alto Adige]>
Biferno Bianco  
<[OK] Arcgis - Geocode [Bianco, Reggio di Calabria]>
Côtes de Provence La Londe  
<[OK] Arcgis - Geocode [Provence Street, London, England, N1 8]>
Monferrato  
<[OK] Arcgis - Geocode [Monferrato]>
Veronese  
<[OK] Arcgis - Geocode [Veronese, Borgo Veneto, Padova]>
Cheverny  
<[OK] Arcgis - Geocode [Cheverny, Loir-et-Cher, Centre-Val de Loire]>
Montecucco  
<[OK] Arcgis - Geocode [Montecucco, Provincia di Brescia]>
Apremont  
<[OK] Arcgis - Geocode

<[OK] Arcgis - Geocode [Coast St, Willits, California, 95490]>
Alcamo  
<[OK] Arcgis - Geocode [Alcamo, Trapani]>
Liguria  
<[OK] Arcgis - Geocode [Liguria]>
Vin de Pays des Coteaux de Peyriac  
<[OK] Arcgis - Geocode [Côteaux, Sud]>
Colli Bolognesi  
<[OK] Arcgis - Geocode [Via dei Colli, 40136, Bologna]>
Ramandolo  
<[OK] Arcgis - Geocode [Ramandolo, Nimis, Udine]>
Siskiyou County California Other
<[OK] Arcgis - Geocode [Siskiyou County, California]>
Charmes-Chambertin  
<[OK] Arcgis - Geocode [Le Charmes-Chambertin]>
Amador-Napa California Other
<[OK] Arcgis - Geocode [Amador County, California]>
Prosecco di Valdobbiadene  
<[OK] Arcgis - Geocode [Valdobbiadene, Treviso]>
Montecarlo  
<[OK] Arcgis - Geocode [Montecarlo]>
Côte Rôtie  
<[OK] Arcgis - Geocode [Côte Rôtie, Le Tourne, Gironde, Nouvelle-Aquitaine]>
Donnici  
<[OK] Arcgis - Geocode [Donnini, Reggello, Firenze]>
Sonoma Valley Sonoma
<[OK] Arcgis - Geocode [Sonoma Valley Tech]>
Vermentino di Gallura  
<[OK] Arcgis - Geocode 

<[OK] Arcgis - Geocode [McLaren Ct, Adelaide, South Australia, 5000]>
Utiel-Requena  
<[OK] Arcgis - Geocode [Travesía Utiel, 46390, Requena, Valencia, Comunidad Valenciana]>
Barbera d'Asti Superiore Nizza  
<[OK] Arcgis - Geocode [Via Nizza, 14047, Mombercelli, Asti]>
Savigny-lès-Beaune  
<[OK] Arcgis - Geocode [Savigny-lès-Beaune, Côte-d'Or, Bourgogne-Franche-Comté]>
Kangaroo Island  
<[OK] Arcgis - Geocode [Kangaroo Island, South Australia]>
Mâcon-Fuissé  
<[OK] Arcgis - Geocode [Route de Fuissé, 71000, Mâcon, Saône-et-Loire, Bourgogne-Franche-Comté]>
Chianti Colli Fiorentini  
<[OK] Arcgis - Geocode [Fiorentini, Casoli, Chieti]>
Pays de Loire  
<[OK] Arcgis - Geocode [Pays de la Loire]>
Toro  
<[OK] Arcgis - Geocode [Toro, Bauchi]>
Madera Central Valley
<[OK] Arcgis - Geocode [Central Ct, Madera, California, 93637]>
Valpolicella Ripasso  
<[OK] Arcgis - Geocode [Valpolicella]>
Missouri  
<[OK] Arcgis - Geocode [Missouri]>
Cerasuolo di Vittoria  
<[OK] Arcgis - Geocode [Via del Cera

...

<[OK] Arcgis - Geocode [Mendecino Way, Morgan Hill, California, 95037]>
Valpolicella  
<[OK] Arcgis - Geocode [Valpolicella]>
Val di Neto  
<[OK] Arcgis - Geocode [Neto, Cachoeirinha, Pernambuco]>
Barbera d'Asti  
<[OK] Arcgis - Geocode [Barrera, Villa San Secondo, Asti]>
Touraine  
<[OK] Arcgis - Geocode [Touraine, Département de l'Indre, Centre]>
Neuquén  
<[OK] Arcgis - Geocode [Neuquén, Confluencia, Neuquén]>
San Marino  
<[OK] Arcgis - Geocode [San Marino]>
Vino de Calidad de Valtiendas  
<[OK] Arcgis - Geocode [Valtiendas, Segovia, Castilla y León]>
Prince Edward County  
<[OK] Arcgis - Geocode [Prince Edward County, Ontario]>
Mendocino-Lake County-Napa North Coast
<[OK] Arcgis - Geocode [Mendocino St, Clearlake, California, 95422]>
Barossa Valley  
<[OK] Arcgis - Geocode [Barossa Valley]>
Friuli Isonzo  
<[OK] Arcgis - Geocode [Isonzo, Rapino, Chieti]>
Moulis-en-Médoc  
<[OK] Arcgis - Geocode [Moulis-en-Médoc, Gironde, Nouvelle-Aquitaine]>
Anderson Valley Mendocino/Lake Counties

<[OK] Arcgis - Geocode [Strada del Recioto, 37024, Negrar di Valpolicella, Verona]>
Nardò  
<[OK] Arcgis - Geocode [Nardò, Lecce]>
Macedon Ranges  
<[OK] Arcgis - Geocode [Macedon Ranges, Victoria]>
Romanée-St.-Vivant  
<[OK] Arcgis - Geocode [Saint-Vivant, Biarne, Jura, Bourgogne-Franche-Comté]>
Campo de Borja  
<[OK] Arcgis - Geocode [Jirón José R. del Campo, Ur. Mexico, La Victoria, Lima, 15034]>
Walla Walla Valley (WA) Columbia Valley
<[OK] Arcgis - Geocode [Columbia St, Walla Walla, Washington, 99362]>
Lodi Central Valley
<[OK] Arcgis - Geocode [Central Ave, Lodi, New Jersey, 07644]>
Coteaux Varois  
<[OK] Arcgis - Geocode [Côteaux, Sud]>
Alella  
<[OK] Arcgis - Geocode [Alella, Barcelona, Catalunya]>
Sonoma County-Monterey County-Santa Barbara County California Other
<[OK] Arcgis - Geocode [Santa Barbara County, California]>
Sonoma County-Monterey County California Other
<[OK] Arcgis - Geocode [Monterey County, California]>
Bergerac Rosé  
<[OK] Arcgis - Geocode [Bergerac]>
Bourg

<[OK] Arcgis - Geocode [Elba, Alabama]>
Asprinio di Aversa  
<[OK] Arcgis - Geocode [Aversa, Caserta]>
Monterrei  
<[OK] Arcgis - Geocode [Monterrei, Monterrey, Nuevo León]>
Sonoma-Napa-Mendocino North Coast
<[OK] Arcgis - Geocode [Mendocino Ave, Santa Rosa, California, 95403]>
Saint-Bris  
<[OK] Arcgis - Geocode [Saint-Brais, Jura]>
Lake Erie Finger Lakes
<[OK] Arcgis - Geocode [Finger Lakes]>
Mâcon-Azé  
<[OK] Arcgis - Geocode [Maconze]>
Campania  
<[OK] Arcgis - Geocode [Campania]>
Murgia  
<[OK] Arcgis - Geocode [Murgia, Zuia, Araba, Euskadi]>
Provincia di Pavia  
<[OK] Arcgis - Geocode [Provincia di Pavia]>
Muscadet  
<[OK] Arcgis - Geocode [Muscadet, Joigny, Yonne, Bourgogne-Franche-Comté]>
Washington Washington Other
<[OK] Arcgis - Geocode [Washington, Utah]>
Castillon Côtes de Bordeaux  
<[OK] Arcgis - Geocode [Rue Castillon, 33000, Bordeaux, Gironde, Nouvelle-Aquitaine]>
Diano d'Alba  
<[OK] Arcgis - Geocode [Diano d'Alba, Cuneo]>
Côtes du Tarn  
<[OK] Arcgis - Geocode [Las Cl

<[OK] Arcgis - Geocode [Fixin, Côte-d'Or, Bourgogne-Franche-Comté]>
Madrid  
<[OK] Arcgis - Geocode [Madrid, Comunidad de Madrid]>
Givry  
<[OK] Arcgis - Geocode [Givry, Saône-et-Loire, Bourgogne-Franche-Comté]>
El Pomar District Central Coast
<[OK] Arcgis - Geocode [Central Coast Regional District, British Columbia]>
Vin de Pays de la Haute Vallée de l’Aude  
<[OK] Arcgis - Geocode [Rue de la Vallée, Ste-Adèle, Québec, J8B]>
Auxey-Duresses  
<[OK] Arcgis - Geocode [Auxey-Duresses, Côte-d'Or, Bourgogne-Franche-Comté]>
Colline Pescaresi  
<[OK] Arcgis - Geocode [Pescăreşti]>
Heathcote  
<[OK] Arcgis - Geocode [Heathcote, Sydney, New South Wales]>
Stags Leap District Napa
<[OK] Arcgis - Geocode [Nappa]>
Vin de Pays d'Oc  
<[OK] Arcgis - Geocode [Le Pays d'Oc]>
Offida Pecorino  
<[OK] Arcgis - Geocode [Pecorino]>
Côte de Beaune  
<[OK] Arcgis - Geocode [La Cote de Beaune]>
Langhe  
<[OK] Arcgis - Geocode [Langhe]>
Old Mission Peninsula  
<[OK] Arcgis - Geocode [Old Mission Peninsula Schoo

<[OK] Arcgis - Geocode [Santa Lucía, Lambaré, Central]>
Pine Mountain-Cloverdale Peak Sonoma
<[OK] Arcgis - Geocode [Pine Mountain Rd, Cloverdale, California, 95425]>
Nevada  
<[OK] Arcgis - Geocode [Nevada]>
Montlouis-sur-Loire  
<[OK] Arcgis - Geocode [Montlouis-sur-Loire, Indre-et-Loire, Centre-Val de Loire]>
Colli di Conegliano  
<[OK] Arcgis - Geocode [Via dei Colli, 31015, Conegliano, Treviso]>
Puglia  
<[OK] Arcgis - Geocode [Puglia]>
Assisi  
<[OK] Arcgis - Geocode [Assisi, Perugia]>
Muscadet Sèvre et Maine Clisson  
<[OK] Arcgis - Geocode [Place du Muscadet, 44190, Clisson, Loire-Atlantique, Pays de la Loire]>
Fronsac  
<[OK] Arcgis - Geocode [Fronsac, Gironde, Nouvelle-Aquitaine]>
Capay Valley Central Valley
<[OK] Arcgis - Geocode [Central Valley]>
Del Veneto  
<[OK] Arcgis - Geocode [Veneto]>
Offida Rosso  
<[OK] Arcgis - Geocode [Rosso, Saint-Louis]>
Anjou Villages  
<[OK] Arcgis - Geocode [Montée du Village, 38150, Anjou, Isère, Auvergne-Rhône-Alpes]>
Trinity County North 

In [21]:
with open('region_coordinates.csv', 'w') as fp:
    fp.write('\n'.join('%s,%f,%f' % x for x in singular_region_complete_with_latlong))

In [25]:
df_region_coordinates = pd.read_csv('region_coordinates.csv')

df = pd.merge(df, df_region_coordinates, on="region_complete")

In [26]:
df.to_csv("competition_dataset/mine_dev.csv",index=False)

In [30]:
df = pd.get_dummies(df, columns=["designation_tr","province","variety"])

In [32]:
df.columns

Index(['country', 'description', 'designation', 'region_1', 'region_2',
       'winery', 'quality', 'region_complete', 'lat', 'long',
       ...
       'variety_Viura', 'variety_Viura-Chardonnay',
       'variety_Viura-Sauvignon Blanc', 'variety_Viura-Verdejo',
       'variety_White Blend', 'variety_White Riesling', 'variety_Xarel-lo',
       'variety_Zibibbo', 'variety_Zinfandel', 'variety_Zweigelt'],
      dtype='object', length=23120)

In [1]:
df.corr()

NameError: name 'df' is not defined